# Run BASiCS for estimation method comparisons

In [1]:
library(Seurat)
# library(SeuratData)
library(cowplot)
library(patchwork)
# library(scHOT)
library(tidyverse)
library(dplyr)
library(BiocParallel)
library(readr)
library(Matrix)

The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, will retire in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
The sp package is now running under evolution status 2
     (status 2 uses the sf package in place of rgdal)

Attaching SeuratObject


Attaching package: ‘patchwork’


The following object is masked from ‘package:cowplot’:

    align_plots


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()    masks stats::filter()
✖ dplyr

In [2]:
library(Seurat)
# library(SeuratData)
library(SeuratDisk)

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat



In [3]:
suppressMessages(library(BASiCS))


### Convert h5ad to seurat objects for simulations

In [4]:
# setwd('/data_volume/memento/simulation/inference/')



# filenames <- c('de','dv')

# for (f in filenames){
#     filename = paste(
#         f,
#         '_filtered.h5ad',
#         sep=''
#     )
#     Convert(filename, dest = "h5seurat", overwrite = TRUE, verbose = TRUE)
        
# }

In [5]:
# setwd('/data_volume/memento/simulation/dv/')

### BASiCS for simulations

In [6]:
setwd('/data_volume/memento/simulation/dv/')


In [7]:
Convert('high_expr_anndata.h5ad', dest = "h5seurat", overwrite = TRUE, verbose = FALSE, misc=FALSE)
seurat <- LoadH5Seurat('high_expr_anndata.h5seurat', meta.data=FALSE,misc=FALSE, verbose = FALSE)
df <- read.table('obs.csv', sep=',', header=TRUE)

seurat$condition <- df$condition
seurat$batch<- df$group
Idents(seurat) <- 'condition'

Warning message:
“Unknown file type: h5ad”
Warning message:
“'assay' not set, setting to 'RNA'”
Creating h5Seurat file for version 3.1.5.9900

Validating h5Seurat file



In [8]:
cond <- 'ctrl'

dat <- subset(x=seurat, idents=cond)
dat.sce <- SingleCellExperiment(
    assays=list(counts = GetAssayData(dat)+1),
    colData = data.frame(BatchInfo = dat$batch))

Chain <- BASiCS_MCMC(
  dat.sce,
  N = 5000, Thin = 100, Burn = 500, WithSpikes = FALSE, SubsetBy = 'cell',
  PrintProgress = FALSE, Regression = TRUE,Threads = getOption("Ncpus", 10),)


Running no spikes BASiCS sampler (regression case) ... 




-----------------------------------------------------
MCMC sampler has been started: 5000 iterations to go.
-----------------------------------------------------
-----------------------------------------------------
End of Burn-in period.
-----------------------------------------------------
 
-----------------------------------------------------
-----------------------------------------------------
All 5000 MCMC iterations have been completed.
-----------------------------------------------------
-----------------------------------------------------
 
-----------------------------------------------------
Please see below a summary of the overall acceptance rates.
-----------------------------------------------------
 
Minimum acceptance rate among mu[i]'s: 0.159333
Average acceptance rate among mu[i]'s: 0.443385
Maximum acceptance rate among mu[i]'s: 0.671556
 
 
Minimum acceptance rate among delta[i]'s: 0.361556
Average acceptance rate among delta[i]'s: 0.480782
Maximum acceptance ra

-------------------------------------------------------------
MCMC running time 
-------------------------------------------------------------
user: 174.789
system: 5.533
elapsed: 18.54


-------------------------------------------------------------
Output 
-------------------------------------------------------------




In [9]:
cond <- 'stim'

dat <- subset(x=seurat, idents=cond)
dat.sce <- SingleCellExperiment(
    assays=list(counts = GetAssayData(dat)+1),
    colData = data.frame(BatchInfo = dat$batch))

Chain2 <- BASiCS_MCMC(
  dat.sce,
  N = 5000, Thin = 100, Burn = 500, WithSpikes = FALSE, SubsetBy = 'cell',
  PrintProgress = FALSE, Regression = TRUE,Threads = getOption("Ncpus", 10),)


Running no spikes BASiCS sampler (regression case) ... 




-----------------------------------------------------
MCMC sampler has been started: 5000 iterations to go.
-----------------------------------------------------
-----------------------------------------------------
End of Burn-in period.
-----------------------------------------------------
 
-----------------------------------------------------
-----------------------------------------------------
All 5000 MCMC iterations have been completed.
-----------------------------------------------------
-----------------------------------------------------
 
-----------------------------------------------------
Please see below a summary of the overall acceptance rates.
-----------------------------------------------------
 
Minimum acceptance rate among mu[i]'s: 0.315333
Average acceptance rate among mu[i]'s: 0.493958
Maximum acceptance rate among mu[i]'s: 0.745778
 
 
Minimum acceptance rate among delta[i]'s: 0.346
Average acceptance rate among delta[i]'s: 0.43772
Maximum acceptance rate a

-------------------------------------------------------------
MCMC running time 
-------------------------------------------------------------
user: 180.134
system: 5.805
elapsed: 19.12


-------------------------------------------------------------
Output 
-------------------------------------------------------------




In [10]:
Test <- BASiCS_TestDE(
  Chain1 = Chain, Chain2 = Chain2,
  GroupLabel1 = "ctrl", GroupLabel2 = "stim",
  EpsilonM = 0, EpsilonD =0,
  EpsilonR = 0.0,
  EFDR_M = NULL, EFDR_D = NULL,EFDR_R=NULL, k=1,ProbThresholdR=0., ProbThresholdM=0, ProbThresholdD=0,
  Offset = TRUE, PlotOffset = TRUE, Plot = TRUE
)
dv <- as.data.frame(Test, Parameter = "ResDisp")
print(dim(dv))

-------------------------------------------------------------
Log-fold change thresholds are now set in a log2 scale. 
Original BASiCS release used a natural logarithm scale.

-------------------------------------------------------------
Offset estimate: 0.4255
(ratio ctrl vs stim).
-------------------------------------------------------------


EFDR = NULL for Differential mean task:
Probability threshold automatically set equal to'ProbThresholdM'.

EFDR = NULL for Differential dispersion task:
Probability threshold automatically set equal to'ProbThresholdD'.

EFDR = NULL for Differential residual dispersion task:
Probability threshold automatically set equal to'ProbThresholdR'.



[1] 3 8


In [30]:
(Test@Results$ResDisp)@Table

,GeneName,ResDispOverall,ResDisp1,ResDisp2,ResDispDistance,ProbDiffResDisp,ResultDiffResDisp,MeanOverall
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
gene10,gene10,0.404,0.243,0.564,-0.376,0.867,ExcludedLowESS,7.542
gene11,gene11,1.178,1.301,1.054,0.238,0.778,ExcludedLowESS,24.708
gene19,gene19,0.042,0.005,0.079,-0.149,0.333,ExcludedLowESS,9.290
gene34,gene34,-0.084,-0.088,-0.080,-0.028,0.111,ExcludedLowESS,12.241
gene37,gene37,-0.206,-0.211,-0.200,-0.020,0.156,ExcludedLowESS,9.274
gene105,gene105,-0.211,-0.195,-0.228,0.041,0.200,ExcludedLowESS,18.155
gene128,gene128,-0.164,-0.186,-0.143,-0.075,0.289,ExcludedLowESS,14.058
gene197,gene197,0.061,0.010,0.113,-0.050,0.156,ExcludedLowESS,8.340
gene253,gene253,-0.056,-0.003,-0.108,0.047,0.156,ExcludedLowESS,11.188


In [ ]:
-------------------------------------------------------------
MCMC running time 
-------------------------------------------------------------
user: 352.699
system: 13.865
elapsed: 37.482

In [ ]:
-------------------------------------------------------------
MCMC running time 
-------------------------------------------------------------
user: 3530.898
system: 365.907
elapsed: 194.038

In [6]:
runBASiCS <- function(seurat, group){
    Idents(seurat) <- 'ct_real'
    dat <- subset(x = seurat, idents = group)
    dat$batch <- sample(1:2, dim(dat)[2], replace=TRUE)
    dat.sce <- SingleCellExperiment(
        assays=list(counts = GetAssayData(dat)),
        colData = data.frame(BatchInfo = dat$batch))
    Chain <- BASiCS_MCMC(
      dat.sce,
      N = 20000, Thin = 20, Burn = 10000, WithSpikes = FALSE, SubsetBy = 'cell',
      PrintProgress = TRUE, Regression = TRUE,Threads = getOption("Ncpus", 42),)
    return(Chain)
}

In [25]:
Chain

An object of class BASiCS_Chain
 90 MCMC samples.
 Dataset contains 986 biological genes and 100 cells (5 batches). 
 Object stored using BASiCS version:  2.12.3 
 Parameters:  mu delta s nu theta beta sigma2 epsilon RefFreq RBFLocations 

In [16]:
library(zellkonverter)

Registered S3 methods overwritten by 'zellkonverter':
  method                                             from      
  py_to_r.numpy.ndarray                              reticulate
  py_to_r.pandas.core.arrays.categorical.Categorical reticulate



In [22]:
df <- read.table('obs.csv', sep=',', header=TRUE)

In [24]:
seurat

An object of class Seurat 
19719 features across 1000 samples within 1 assay 
Active assay: RNA (19719 features, 0 variable features)

In [ ]:
f <- 'dv'
print(f)

infilename = paste(
    f,
    '_filtered.h5seurat',
    sep=''
)

data_obj <- LoadH5Seurat(infilename, misc=F, verbose = FALSE)
print(dim(data_obj))

A_chain <- runBASiCS(data_obj, 'A')
saveRDS(A_chain, file = paste(f, "_A_filtered_chain.rds", sep=""))
B_chain <- runBASiCS(data_obj, 'B')
saveRDS(B_chain, file = paste(f, "_B_filtered_chain.rds", sep=""))


[1] "dv"


Validating h5Seurat file



[1] 1625 2000


Running no spikes BASiCS sampler (regression case) ... 




In [45]:
A_chain <- readRDS(paste('dv', "_A_filtered_chain.rds", sep=""))
B_chain <- readRDS(paste('dv', "_B_filtered_chain.rds", sep=""))

Test <- BASiCS_TestDE(
  Chain1 = A_chain, Chain2 = B_chain,
  GroupLabel1 = "A", GroupLabel2 = "B",
  EpsilonM = 0, EpsilonD =0,
  EpsilonR = 0,
  EFDR_M = 1, EFDR_D = 1,EFDR_R=NULL, k=1,ProbThresholdR=0,
  Offset = TRUE, PlotOffset = TRUE, Plot = TRUE
)

dv <- as.data.frame(Test, Parameter = "ResDisp")

-------------------------------------------------------------
Log-fold change thresholds are now set in a log2 scale. 
Original BASiCS release used a natural logarithm scale.

-------------------------------------------------------------
Offset estimate: 0.9902
(ratio A vs B).
-------------------------------------------------------------


For Differential mean task:
the posterior probability threshold chosen via EFDR calibrationis too low. Probability threshold automatically set equal to'ProbThresholdM'.

For Differential dispersion task:
the posterior probability threshold chosen via EFDR calibrationis too low. Probability threshold automatically set equal to'ProbThresholdD'.

EFDR = NULL forDifferential residual dispersion task:
Probability threshold automatically set equal to'ProbThresholdR'.



In [47]:
write_csv(dv, file = 'dv_basics.csv')


### run BASiCS for smFISH comparison

In [3]:
setwd('/data_volume/memento/saver/')
data_path <- '/data_volume/memento/saver/'

In [10]:
Convert('dropseq.h5ad', dest = "h5seurat", overwrite = TRUE, verbose = FALSE)

Warning message:
“Unknown file type: h5ad”
Warning message:
“'assay' not set, setting to 'RNA'”
Creating h5Seurat file for version 3.1.5.9900



In [12]:
runBASiCS_dropseq <- function(seurat, stat){
    Idents(seurat) <- 'group'
    dat <- seurat
    dat$batch <- 1
    dat.sce <- SingleCellExperiment(
    assays=list(counts = GetAssayData(dat)),
    colData = data.frame(BatchInfo = dat$batch))
    Chain <- BASiCS_MCMC(
      dat.sce,
      N = 1000, Thin = 10, Burn = 500, WithSpikes = FALSE, SubsetBy = 'cell',
      PrintProgress = FALSE, Regression = TRUE,Threads = getOption("Ncpus", 10),)
    df = as.data.frame(displayChainBASiCS(Chain, Param = stat))
    return(df)
}

In [10]:
data_obj <- LoadH5Seurat('dropseq.h5seurat', verbose = FALSE)

mu <- runBASiCS_dropseq(data_obj, 'mu')
write_csv(mu, file = 'dropseq_BASiCS_mu.csv')

delta <- runBASiCS_dropseq(data_obj, 'delta')
write_csv(delta, file = 'dropseq_BASiCS_delta.csv')

ERROR: Error: Cannot find file dropseq.h5seurat
